# **Preparation Notebook**



---
## Setup Environment

In [1]:
# DO NOT MODIFY THE CODE IN THIS CELL
!pip install -q utstd

from utstd.folders import *
from utstd.ipyrenders import *

at = AtFolder(
    course_code=36106,
    assignment="AT3",
)
at.run()

import warnings
warnings.simplefilter(action='ignore')


[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



You can now save your data files in: c:\Users\brohao\Desktop\UTS\36106\ClassificationAnalysis-36106-GroupAssignment\36106\assignment\AT3\data


---
## Student Information

In [ ]:
# <Student to fill this section and then remove this comment>
group_name = "12"
student_name = "Jiayu Hao"
student_id = "25948860"

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h1", key='group_name', value=group_name)

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h1", key='student_name', value=student_name)

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h1", key='student_id', value=student_id)

---
## 0. Python Packages

### 0.a Install Additional Packages

> If you are using additional packages, you need to install them here using the command: `! pip install <package_name>`

In [ ]:
# <Student to fill this section and then remove this comment>

### 0.b Import Packages

In [2]:
# DO NOT MODIFY THE CODE IN THIS CELL
import pandas as pd
import altair as alt

In [8]:
import numpy as np

---
## A. Feature Selection


## A.0 Load Data

In [4]:
# DO NOT MODIFY THE CODE IN THIS CELL
# Load datasets
try:
  sales_2022_df = pd.read_csv(at.folder_path / "sales_2022.csv")
  products_df = pd.read_csv(at.folder_path / "products.csv")
  product_subcats_df = pd.read_csv(at.folder_path / "product_subcats.csv")
  territories_df = pd.read_csv(at.folder_path / "territories.csv")
  sales_2021_df = pd.read_csv(at.folder_path / "sales_2021.csv")
  returns_df = pd.read_csv(at.folder_path / "returns.csv")
  sales_2020_df = pd.read_csv(at.folder_path / "sales_2020.csv")
  product_cats_df = pd.read_csv(at.folder_path / "product_cats.csv")
  customers_df = pd.read_csv(at.folder_path / "customers.csv")
except Exception as e:
  print(e)

In [5]:
# === Quick structural exploration for remaining datasets ===

datasets = {
    "sales_2020": sales_2020_df,
    "sales_2021": sales_2021_df,
    "product_subcats": product_subcats_df,
    "product_cats": product_cats_df,
    "territories": territories_df,
    "returns": returns_df,
    "customers": customers_df
}

for name, df in datasets.items():
    print("=" * 80)
    print(f"Dataset: {name}")
    print("Shape of dataset:", df.shape)
    
    # Basic info
    print("\nDataset Info:")
    print(df.info())
    
    # Preview first few rows
    print("\nFirst 5 rows:")
    pd.set_option("display.max_columns", None)
    display(df.head())
    
    # Summary statistics for numeric columns
    print("\nSummary statistics for numerical variables:")
    if len(df.select_dtypes(include=["number"]).columns) > 0:
        display(df.describe().T)
    else:
        print("No numeric columns found.")
    
    # Unique value count for categorical variables
    print("\nUnique values count for categorical variables:")
    categorical_cols = df.select_dtypes(include=["object", "category"]).columns
    if len(categorical_cols) > 0:
        for col in categorical_cols:
            print(f"{col}: {df[col].nunique()} unique values")
    else:
        print("No categorical columns found.")
    
    # Missing and duplicates check
    print("\nMissing values per column:")
    missing = df.isnull().sum()
    missing = missing[missing > 0].sort_values(ascending=False)
    display(missing if not missing.empty else "No missing values.")
    
    print("Number of duplicate rows:", df.duplicated().sum())
    print("=" * 80, "\n")

 Dataset: sales_2020
Shape of dataset: (4153, 9)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4153 entries, 0 to 4152
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     4153 non-null   int64  
 1   OrderDate      4153 non-null   object 
 2   StockDate      3107 non-null   object 
 3   OrderNumber    4055 non-null   object 
 4   ProductKey     4153 non-null   int64  
 5   CustomerKey    4153 non-null   int64  
 6   TerritoryKey   3978 non-null   float64
 7   OrderLineItem  3446 non-null   float64
 8   OrderQuantity  4057 non-null   float64
dtypes: float64(3), int64(3), object(3)
memory usage: 292.1+ KB
None

First 5 rows:


,Unnamed: 0,OrderDate,StockDate,OrderNumber,ProductKey,CustomerKey,TerritoryKey,OrderLineItem,OrderQuantity
0,0,2020-02-23,2019-11-01,SO45478,313,29354,1.0,NaN,1.0
1,1,2020-03-22,2020-02-05,SO45719,350,20000,6.0,1.0,1.0
2,2,2020-05-12,2020-02-22,SO46185,314,15632,10.0,1.0,1.0
3,3,2020-07-04,2020-05-28,SO46705,370,13085,1.0,1.0,1.0
4,4,2020-12-24,2020-11-12,SO48661,332,15467,4.0,1.0,1.0



Summary statistics for numerical variables:


,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,4153.0,2076.000000,1199.012163,0.0,1038.0,2076.0,3114.0,4152.0
ProductKey,4153.0,343.699013,26.219733,310.0,314.0,347.0,369.0,389.0
CustomerKey,4153.0,18285.025283,5551.774487,11090.0,13118.0,16561.0,22572.0,29481.0
TerritoryKey,3978.0,6.572901,2.886477,1.0,4.0,8.0,9.0,10.0
OrderLineItem,3446.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
OrderQuantity,4057.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0



Unique values count for categorical variables:
OrderDate: 365 unique values
StockDate: 441 unique values
OrderNumber: 2589 unique values

Missing values per column:


...


Summary statistics for numerical variables:


,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,19969.0,9984.000000,5764.698098,0.0,4992.0,9984.0,14976.0,19968.0
annual_income,19963.0,57245.904924,32188.002748,10000.0,30000.0,60000.0,70000.0,170000.0
number_dependents,19963.0,1.840054,1.615045,0.0,0.0,2.0,3.0,5.0



Unique values count for categorical variables:
customer_id: 19967 unique values
prefix: 4 unique values
first_name: 666 unique values
last_name: 372 unique values
birth_date: 8191 unique values
marital_status: 2 unique values
email_address: 18148 unique values
education_level: 5 unique values
occupation: 5 unique values
homeowner: 2 unique values

Missing values per column:


prefix               146
first_name             6
last_name              6
marital_status         6
email_address          6
annual_income          6
number_dependents      6
occupation             6
education_level        6
homeowner              6
birth_date             3
dtype: int64

Number of duplicate rows: 0



### A.1 Approach 1

In [6]:
# === A.1 Select business-relevant features ===
# Keep only attributes useful for customer-level feature engineering.

print("A.1 Selecting relevant columns...")

products_cols = [
    'product_id','product_subcategory_id','product_style',
    'product_price','product_cost'
]
sales_cols = [
    'orderdate','order_number','product_id','customer_key',
    'territorykey','orderlineitem','orderquantity'
]
customers_cols = [
    'customer_key','age','annual_income','number_dependents',
    'education_level','occupation','marital_status','homeowner'
]

print("Features selected from products, sales, and customers.")

A.1 Selecting relevant columns...
Features selected from products, sales, and customers.


In [ ]:
# <Student to fill this section and then remove this comment>
feature_selection_1_insights = """
provide an explanation on why you use this approach for feature selection and describe its results
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_1_insights', value=feature_selection_1_insights)

### A.2 Approach 2

In [ ]:
# <Student to fill this section>
feature_selection_2_insights = """
provide an explanation on why you use this approach for feature selection and describe its results
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_2_insights', value=feature_selection_2_insights)

### A.n Approach "\<describe_approach_here\>"

> You can add more cells related to other approaches in this section

In [ ]:
# <Student to fill this section and then remove this comment>

In [ ]:
# <Student to fill this section>
feature_selection_n_insights = """
provide an explanation on why you use this approach for feature selection and describe its results
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_n_insights', value=feature_selection_n_insights)

### A.z Final Selection of Features

In [ ]:
# <Student to fill this section and then remove this comment>

features_list = []

In [ ]:
# === A.z Final Selection of Features ===
# We will build customer-level features (Q1 features → Q2 labels).
# NOTE: sales_2022 covers Jan–Jun. To simulate "next 3 months", use:
#   Feature window: 2022-01-01 ~ 2022-03-31 (Q1)
#   Label window:   2022-04-01 ~ 2022-06-30 (Q2)

print("A.z Final feature list prepared for customer-level aggregation...")

features_list = [
    # Customer behavior (Q1)
    'freq_q1', 'qty_sum_q1', 'n_orders_q1', 'n_products_q1',
    'recency_days_q1',
    # Product economics (via join)
    'revenue_sum_q1', 'avg_price_q1', 'avg_cost_q1', 'avg_margin_rate_q1',
    # Diversity
    'n_subcats_q1', 'n_cats_q1',
    # Territory signal
    'major_territory_q1'
]

print("Features:", features_list)

In [ ]:
# <Student to fill this section and then remove this comment>
feature_selection_explanations = """
provide a quick explanation on the features selected
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_explanations', value=feature_selection_explanations)

---
## B. Data Cleaning

### B.1 Fixing "\<describe_issue_here\>"

In [ ]:
# <Student to fill this section and then remove this comment>

In [9]:
# === B.1 Drop redundant columns & harmonize naming ===
print("B.1 Cleaning datasets...")

def drop_unnamed(df):
    return df[[c for c in df.columns if not c.lower().startswith('unnamed')]]

# Products
products = drop_unnamed(products_df).rename(columns=str.lower)
# Add missing economics
products['margin_rate'] = (
    products['product_price'] - products['product_cost']
) / products['product_price']
products['log_price'] = np.log(products['product_price']).replace(-np.inf, np.nan)
products['log_cost']  = np.log(products['product_cost']).replace(-np.inf, np.nan)

# Sales 2022 already clean; harmonize 2020–2021
s20 = sales_2020_df.rename(columns={
    'OrderDate':'orderdate','StockDate':'stockdate','OrderNumber':'order_number',
    'ProductKey':'product_id','CustomerKey':'customer_key',
    'TerritoryKey':'territorykey','OrderLineItem':'orderlineitem','OrderQuantity':'orderquantity'
})
s21 = sales_2021_df.rename(columns={
    'order_date':'orderdate','stock_date':'stockdate','order_number':'order_number',
    'product_key':'product_id','customer_key':'customer_key',
    'territory_key':'territorykey','order_line_item':'orderlineitem','order_quantity':'orderquantity'
})

# Unify formats
for df_ in (s20, s21, sales_2022_df):
    df_ = drop_unnamed(df_)
    df_['orderdate'] = pd.to_datetime(df_['orderdate'], errors='coerce')
    df_['stockdate'] = pd.to_datetime(df_['stockdate'], errors='coerce')

print("Datasets harmonized.")

B.1 Cleaning datasets...
Datasets harmonized.


In [ ]:
# <Student to fill this section and then remove this comment>
data_cleaning_1_explanations = """
Provide some explanations on why you believe it is important to fix this issue and its impacts
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_1_explanations', value=data_cleaning_1_explanations)

### B.2 Fixing "\<describe_issue_here\>"

In [ ]:
# <Student to fill this section and then remove this comment>

In [10]:
# === B.2 Product hierarchy & territories ===
print("B.2 Linking product and territory metadata...")

subcats = product_subcats_df.rename(columns={
    'product_subcategory_key':'product_subcategory_id',
    'subcategory_name':'subcategory_name',
    'product_category_id':'product_category_id'
})
cats = product_cats_df.rename(columns={
    'ProductCategoryKey':'product_category_id',
    'CategoryName':'category_name'
})
prod_hierarchy = subcats.merge(cats, on='product_category_id', how='left')
products = products.merge(prod_hierarchy[['product_subcategory_id','category_name']],
                          on='product_subcategory_id', how='left')

territories = territories_df.rename(columns={
    'SalesTerritoryKey':'territorykey',
    'Region':'region','Country':'country','Continent':'continent'
})
print("Metadata attached.")

B.2 Linking product and territory metadata...
Metadata attached.


In [ ]:
# <Student to fill this section and then remove this comment>
data_cleaning_2_explanations = """
Provide some explanations on why you believe it is important to fix this issue and its impacts
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_2_explanations', value=data_cleaning_2_explanations)

### B.3 Fixing "\<describe_issue_here\>"

In [ ]:
# <Student to fill this section and then remove this comment>

In [11]:
# === B.3 Returns cleaning ===
print("B.3 Cleaning returns data...")
returns = drop_unnamed(returns_df).rename(columns={
    'return_date':'return_date','product_key':'product_id',
    'territory_id':'territorykey','return_quantity':'return_quantity'
})
returns['return_date'] = pd.to_datetime(returns['return_date'], errors='coerce')
# Aggregate returns by product & territory
ret_prod = returns.groupby('product_id')['return_quantity'].sum().reset_index()
ret_terr = returns.groupby('territorykey')['return_quantity'].sum().reset_index()
print("Returns aggregates ready.")

B.3 Cleaning returns data...
Returns aggregates ready.


In [ ]:
# <Student to fill this section and then remove this comment>
data_cleaning_3_explanations = """
Provide some explanations on why you believe it is important to fix this issue and its impacts
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_3_explanations', value=data_cleaning_3_explanations)

### B.n Fixing "\<describe_issue_here\>"

> You can add more cells related to other issues in this section

In [33]:
# === B.4 Clean customers (from raw) ===
print("B.4 Cleaning customers data...")

customers = drop_unnamed(customers_df).copy()
customers.columns = customers.columns.str.strip().str.lower().str.replace(" ", "_")

# Fix types
customers['birth_date'] = pd.to_datetime(customers['birth_date'], errors='coerce')
for c in ['annual_income','number_dependents']:
    customers[c] = pd.to_numeric(customers[c], errors='coerce')

# Compute age
ref_date = pd.Timestamp("2022-12-31")
customers['age'] = ((ref_date - customers['birth_date']).dt.days // 365.25).astype('Int64')

# Normalize categories
for c in ['education_level','occupation','marital_status','homeowner']:
    customers[c] = customers[c].astype(str).str.strip().str.title().replace("Nan", np.nan)

B.4 Cleaning customers data...


In [34]:

# --- Ensure mapping columns + values align, then merge safely ---
print("\n[ID MAP] Normalising and merging id_mapping...")

# 1) Normalise column names
id_map = pd.read_csv(at.folder_path / "id_mapping.csv")
id_map.columns = id_map.columns.str.strip().str.lower()
id_map = id_map.rename(columns={'customerkey': 'customer_key'})

# 2) Normalise JOIN VALUES (very important)
customers['customer_id'] = customers['customer_id'].astype(str).str.strip().str.lower()
id_map['customer_id']   = id_map['customer_id'].astype(str).str.strip().str.lower()

# 3) If mapping accidentally has other variants, coerce them
if 'customer_key' not in id_map.columns:
    # Fallback: look for any column with 'key' in its name
    key_guess = [c for c in id_map.columns if 'key' in c]
    if key_guess:
        id_map = id_map.rename(columns={key_guess[0]: 'customer_key'})

# 4) Merge
before_cols = customers.columns.tolist()
customers = customers.merge(id_map[['customer_id','customer_key']].drop_duplicates(), 
                            on='customer_id', how='left')
print("Merged customers; new columns added:", [c for c in customers.columns if c not in before_cols])

# 5) Verify presence and show coverage
print("Has 'customer_key' column?:", 'customer_key' in customers.columns)
if 'customer_key' in customers.columns:
    print("Rows with missing customer_key after merge:", customers['customer_key'].isna().sum())

# 6) Guarded selection to avoid KeyError; report missing columns explicitly
customers_cols_expected = ['customer_key','age','annual_income','number_dependents',
                           'education_level','occupation','marital_status','homeowner']
available_cols = [c for c in customers_cols_expected if c in customers.columns]
missing_cols = sorted(set(customers_cols_expected) - set(available_cols))
print("Missing expected columns:", missing_cols)

# Only proceed to drop_duplicates if 'customer_key' exists
if 'customer_key' in customers.columns:
    customers = customers[available_cols].drop_duplicates('customer_key', keep='first')
    print("Customers cleaned:", customers.shape)
else:
    raise KeyError("customer_key is still missing after merge. Check id_mapping column names/values.")


[ID MAP] Normalising and merging id_mapping...
Merged customers; new columns added: ['customer_key']
Has 'customer_key' column?: True
Rows with missing customer_key after merge: 1816
Missing expected columns: []
Customers cleaned: (18152, 8)


In [35]:
print(customers.head(5))

  customer_key  age  annual_income  number_dependents  education_level  \
0          NaN   56        60000.0                0.0  Graduate Degree   
1        19893   70        60000.0                2.0      High School   
2        17574   53        50000.0                2.0        Bachelors   
3        25207   73        30000.0                4.0      High School   
4        27736   45        40000.0                0.0      High School   

       occupation marital_status homeowner  
0  Skilled Manual              S         Y  
1    Professional              S         Y  
2  Skilled Manual              S         N  
3    Professional              S         Y  
4  Skilled Manual              S         Y  


In [31]:
# Check overlap of customer_key between customers and sales_2022
sales_2022_df['customer_key'] = pd.to_numeric(sales_2022_df['customer_key'], errors='coerce').astype('Int64')
customers['customer_key'] = pd.to_numeric(customers['customer_key'], errors='coerce').astype('Int64')
overlap = sales_2022_df['customer_key'].isin(customers['customer_key']).mean()
print(f"Coverage of customers in sales_2022: {overlap:.2%}")

Coverage of customers in sales_2022: 100.00%


In [ ]:
# <Student to fill this section and then remove this comment>
data_cleaning_n_explanations = """
Provide some explanations on why you believe it is important to fix this issue and its impacts
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_n_explanations', value=data_cleaning_n_explanations)

---
## C. Feature Engineering

In [36]:
# DO NOT MODIFY THE CODE IN THIS CELL
# Create copy of datasets

try:
  training_df_eng = training_df_clean.copy()
  validation_df_eng = validation_df_clean.copy()
  testing_df_eng = testing_df_clean.copy()
except Exception as e:
  print(e)

name 'training_df_clean' is not defined


In [ ]:
# An unuseful code part for me.

### C.1 New Feature "\<put_name_here\>"



In [ ]:
# <Student to fill this section and then remove this comment>

In [37]:
# === C.1 Combine multi-year sales ===
print("C.1 Combining sales 2020–2022...")
sales_all = pd.concat([s20, s21, sales_2022_df], ignore_index=True)
sales_all = drop_unnamed(sales_all)
sales_all = sales_all.dropna(subset=['orderdate','customer_key','product_id'])
sales_all['orderquantity'] = sales_all['orderquantity'].fillna(1).astype(int)
print("Total sales records:", sales_all.shape)


C.1 Combining sales 2020–2022...
Total sales records: (87244, 12)


In [ ]:
# <Student to fill this section and then remove this comment>
feature_engineering_1_explanations = """
Provide some explanations on why you believe it is important to create this feature and its impacts
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_1_explanations', value=feature_engineering_1_explanations)

### C.3 New Feature "\<put_name_here\>"



In [ ]:
# <Student to fill this section and then remove this comment>

In [38]:
# === C.2 Join products and returns ===
print("C.2 Adding product and return info...")
sales_all = sales_all.merge(products, on='product_id', how='left')
sales_all = sales_all.merge(ret_prod, on='product_id', how='left')
sales_all = sales_all.merge(ret_terr, on='territorykey', how='left')
print("Sales enriched:", sales_all.shape)


C.2 Adding product and return info...
Sales enriched: (134692, 28)


In [ ]:
# <Student to fill this section and then remove this comment>
feature_engineering_2_explanations = """
Provide some explanations on why you believe it is important to create this feature and its impacts
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_2_explanations', value=feature_engineering_2_explanations)

### C.4 New Feature "\<put_name_here\>"



In [ ]:
# <Student to fill this section and then remove this comment>

In [47]:
# === C.3 Build customer-level aggregates (Jan–Mar 2022 window) ===
print("C.3 Aggregating customer-level features...")

feature_window = (sales_all['orderdate'] >= '2022-01-01') & (sales_all['orderdate'] <= '2022-03-31')
sales_f = sales_all.loc[feature_window].copy()

agg = (sales_f.groupby('customer_key')
       .agg(freq=('order_number','nunique'),
            qty_sum=('orderquantity','sum'),
            n_products=('product_id','nunique'),
            revenue_sum=('product_price','sum'),
            avg_price=('product_price','mean'),
            avg_margin=('margin_rate','mean'),
            returns_total=('return_quantity_x','sum'))
       .reset_index())

print("Customer features shape:", agg.shape)


C.3 Aggregating customer-level features...
Customer features shape: (5140, 8)


In [ ]:
# <Student to fill this section and then remove this comment>
feature_engineering_3_explanations = """
Provide some explanations on why you believe it is important to create this feature and its impacts
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_3_explanations', value=feature_engineering_3_explanations)

### C.4 New Feature "\<put_name_here\>"



In [ ]:
# <Student to fill this section and then remove this comment>

In [51]:
# === C.4 Merge demographics and historical stats ===
print("C.4 Adding demographics and historical aggregates...")

# Ensure consistent key types before merging
agg['customer_key'] = pd.to_numeric(agg['customer_key'], errors='coerce').astype('Int64')
customers['customer_key'] = pd.to_numeric(customers['customer_key'], errors='coerce').astype('Int64')

dataset = agg.merge(customers, on='customer_key', how='left')
# Optional: yearly order counts
hist20 = s20.groupby('customer_key')['order_number'].nunique().reset_index(name='orders_2020')
hist21 = s21.groupby('customer_key')['order_number'].nunique().reset_index(name='orders_2021')
dataset = dataset.merge(hist20, on='customer_key', how='left')
dataset = dataset.merge(hist21, on='customer_key', how='left')
dataset[['orders_2020','orders_2021']] = dataset[['orders_2020','orders_2021']].fillna(0)
dataset['orders_trend'] = dataset['orders_2021'] - dataset['orders_2020']

print("Merged dataset:", dataset.shape)



C.4 Adding demographics and historical aggregates...
Merged dataset: (5140, 18)


In [ ]:
# <Student to fill this section and then remove this comment>
feature_engineering_3_explanations = """
Provide some explanations on why you believe it is important to create this feature and its impacts
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_3_explanations', value=feature_engineering_3_explanations)

### C.n Fixing "\<describe_issue_here\>"

> You can add more cells related to new features in this section

In [ ]:
# <Student to fill this section and then remove this comment>

In [52]:
# === C.5 Create label: purchase_next_3m ===
print("C.5 Creating label (Apr–Jun 2022 purchase)...")

label_window = (sales_all['orderdate'] >= '2022-04-01') & (sales_all['orderdate'] <= '2022-06-30')
future_orders = (sales_all[label_window].groupby('customer_key')['order_number']
                 .nunique().reset_index(name='future_orders'))
future_orders['purchase_next_3m'] = (future_orders['future_orders'] > 0).astype(int)
dataset = dataset.merge(future_orders[['customer_key','purchase_next_3m']], on='customer_key', how='left')
dataset['purchase_next_3m'] = dataset['purchase_next_3m'].fillna(0).astype(int)
print("Label created:", dataset['purchase_next_3m'].mean().round(3))


C.5 Creating label (Jul–Sep 2022 purchase)...
Label created: 0.111


In [50]:
#两段结果一样，只留一个好看的
print("C.5 Creating label (Apr–Jun 2022 purchase)...")

# Ensure datetime type
sales_2022_df['orderdate'] = pd.to_datetime(sales_2022_df['orderdate'], errors='coerce')

# Check date range
print("Date range in sales_2022:", sales_2022_df['orderdate'].min(), "→", sales_2022_df['orderdate'].max())

# Adjust cutoff (data only to June)
cutoff_start = pd.Timestamp("2022-04-01")
cutoff_end = pd.Timestamp("2022-06-30")

# Build future purchase flag
future_purchases = (
    sales_2022_df[
        (sales_2022_df["orderdate"] >= cutoff_start)
        & (sales_2022_df["orderdate"] <= cutoff_end)
    ]
    .groupby("customer_key", as_index=False)
    .agg(future_orders=("order_number", "nunique"))
)

print("Customers with future purchases:", len(future_purchases))
print("Future purchases columns:", future_purchases.columns.tolist())

# Ensure both keys have same dtype
dataset['customer_key'] = pd.to_numeric(dataset['customer_key'], errors='coerce').astype('Int64')
future_purchases['customer_key'] = pd.to_numeric(future_purchases['customer_key'], errors='coerce').astype('Int64')

# Merge safely
dataset = dataset.merge(future_purchases, on="customer_key", how="left")

# If merge fails to bring the column, set missing as 0
if "future_orders" not in dataset.columns:
    dataset["future_orders"] = 0

# Create label
dataset["label"] = (dataset["future_orders"] > 0).astype(int)

print("Label created:", dataset["label"].mean())


C.5 Creating label (Jul–Sep 2022 purchase)...
Date range in sales_2022: 2022-01-01 00:00:00 → 2022-06-30 00:00:00
Customers with future purchases: 5934
Future purchases columns: ['customer_key', 'future_orders']
Label created: 0.111284046692607


In [ ]:
# <Student to fill this section and then remove this comment>
feature_engineering_n_explanations = """
Provide some explanations on why you believe it is important to create this feature and its impacts
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_n_explanations', value=feature_engineering_n_explanations)

---
## D. Data Preparation for Modeling

### D.1 Split Datasets


In [ ]:
# <Student to fill this section>

In [53]:
# === D.1 Handle missing & split data ===
print("D.1 Cleaning final dataset and splitting...")

num_cols = dataset.select_dtypes(include=['number']).columns.drop(['purchase_next_3m'], errors='ignore')
for c in num_cols:
    if dataset[c].isna().mean() < 0.1:
        dataset[c] = dataset[c].fillna(dataset[c].median())

from sklearn.model_selection import train_test_split
X = dataset.drop(columns=['purchase_next_3m'])
y = dataset[['purchase_next_3m']]

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)


D.1 Cleaning final dataset and splitting...
Train: (3598, 18) Val: (771, 18) Test: (771, 18)


In [ ]:
# <Student to fill this section>
data_splitting_explanations = """
Provide some explanations on what is the best strategy to use for data splitting for this dataset
"""

In [ ]:
# Do not modify this code
print_tile(size="h3", key='data_splitting_explanations', value=data_splitting_explanations)

### D.2 Data Transformation <put_name_here>


In [ ]:
# <Student to fill this section and then remove this comment>

In [54]:
# === D.2 Final ready-to-save features ===
print("D.2 Final dataset summary:")
print("Columns:", list(X_train.columns))
print("Target positive rate:", y_train.mean().round(3).values)

D.2 Final dataset summary:
Columns: ['customer_key', 'freq', 'qty_sum', 'n_products', 'revenue_sum', 'avg_price', 'avg_margin', 'returns_total', 'age', 'annual_income', 'number_dependents', 'education_level', 'occupation', 'marital_status', 'homeowner', 'orders_2020', 'orders_2021', 'orders_trend']
Target positive rate: [0.111]


In [ ]:
# <Student to fill this section and then remove this comment>
data_transformation_1_explanations = """
Provide some explanations on why you believe it is important to perform this data transformation and its impacts
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_transformation_1_explanations', value=data_transformation_1_explanations)

### D.3 Data Transformation <put_name_here>

In [ ]:
# <Student to fill this section>

In [ ]:
# <Student to fill this section and then remove this comment>
data_transformation_2_explanations = """
Provide some explanations on why you believe it is important to perform this data transformation and its impacts
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_transformation_2_explanations', value=data_transformation_2_explanations)

### D.4 Data Transformation <put_name_here>


In [ ]:
# <Student to fill this section and then remove this comment>

In [ ]:
# <Student to fill this section and then remove this comment>
data_transformation_3_explanations = """
Provide some explanations on why you believe it is important to perform this data transformation and its impacts
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_transformation_3_explanations', value=data_transformation_3_explanations)

### D.n Fixing "\<describe_issue_here\>"

> You can add more cells related to data preparation in this section

In [ ]:
# <Student to fill this section and then remove this comment>

In [ ]:
# <Student to fill this section and then remove this comment>
data_transformation_n_explanations = """
Provide some explanations on why you believe it is important to perform this data transformation and its impacts
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_transformation_n_explanations', value=data_transformation_n_explanations)

---
## E. Save Datasets

> Do not change this code

In [55]:
# DO NOT MODIFY THE CODE IN THIS CELL

try:
  X_train.to_csv(at.folder_path / 'X_train.csv', index=False)
  y_train.to_csv(at.folder_path / 'y_train.csv', index=False)

  X_val.to_csv(at.folder_path / 'X_val.csv', index=False)
  y_val.to_csv(at.folder_path / 'y_val.csv', index=False)

  X_test.to_csv(at.folder_path / 'X_test.csv', index=False)
  y_test.to_csv(at.folder_path / 'y_test.csv', index=False)
except Exception as e:
  print(e)